# **kaggle比赛**

 泰坦尼克

In [1]:
import pandas as pd
train=pd.read_csv('train1.csv')
test=pd.read_csv('test1.csv')

In [2]:
print(train.info())
print(test.info())
#存在缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null 

人工选择特征

In [3]:
feature=['Pclass','Sex','Age','Embarked','SibSp','Parch','Fare']

In [4]:
X_train=train[feature]
X_test=test[feature]

In [5]:
y_train=train['Survived']

In [6]:
#由于存在缺失值，需要进行补充，于是利用出现次数最多的进行补充
print(X_train['Embarked'].value_counts())
print(X_test['Embarked'].value_counts())

S    644
C    168
Q     77
Name: Embarked, dtype: int64
S    270
C    102
Q     46
Name: Embarked, dtype: int64


In [7]:
X_train['Embarked'].fillna('S',inplace=True)
X_test['Embarked'].fillna('S',inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [8]:
#对于Age存在缺失值，利用均值补充
X_train['Age'].fillna(X_train['Age'].mean(),inplace=True)
X_test['Age'].fillna(X_test['Age'].mean(),inplace=True)

In [9]:
X_test['Fare'].fillna(X_test['Fare'].mean(),inplace=True)

利用Dictvectorizer进行特征工程

In [10]:
from sklearn.feature_extraction import DictVectorizer
vec=DictVectorizer(sparse=False)
X_train=vec.fit_transform(X_train.to_dict(orient='record'))

In [11]:
X_test=vec.transform(X_test.to_dict(orient='record'))

In [12]:
vec.feature_names_

['Age',
 'Embarked=C',
 'Embarked=Q',
 'Embarked=S',
 'Fare',
 'Parch',
 'Pclass',
 'Sex=female',
 'Sex=male',
 'SibSp']

随机森林和XGboost

In [13]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()

In [15]:
from xgboost import XGBClassifier
xgbc=XGBClassifier()

In [16]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(rfc,X_train,y_train,cv=5).mean())
print(cross_val_score(xgbc,X_train,y_train,cv=5).mean())

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

0.8036508431310592
0.81824559798311


In [21]:
#使用默认的随机森林进行预测
rfc.fit(X_train,y_train)
rfc_predict=rfc.predict(X_test)
rfc_submission=pd.DataFrame({'passengerId':test['PassengerId'],'Survived':rfc_predict})
rfc_submission.to_csv('rfc_submission.csv',index=False)

In [25]:
#使用默认的XGBClassifier进行预测
xgbc.fit(X_train,y_train)
xgbc_predict=xgbc.predict(X_test)
xgbc_submission=pd.DataFrame({'passengerId':test['PassengerId'],'Survived':xgbc_predict})
xgbc_submission.to_csv('xgbc_submission.csv',index=False)

使用并行网格搜索，期望得到更好的结果，提高XGBC的预测能力

In [27]:
from sklearn.model_selection import GridSearchCV

In [31]:
params={'max_depth':range(3,7),'n_estimators':range(100,1100,200),'learning_rate':[0.05,0.1,0.25,0.5,1.0]}

In [32]:
xgbc_best=XGBClassifier()

In [33]:
gs=GridSearchCV(xgbc_best,params,n_jobs=-1,cv=5,verbose=1)
gs.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   33.2s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.4min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': range(3, 7), 'n_estimators': range(100, 1100, 200), 'learning_rate': [0.05, 0.1, 0.25, 0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [34]:
print(gs.best_score_)
print(gs.best_params_)

0.835016835016835
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


In [35]:
xgbc_best_predict=gs.predict(X_test)
xgbc_best_submission=pd.DataFrame({'passengerId':test['PassengerId'],'Survived':xgbc_best_predict})
xgbc_best_submission.to_csv('xgbc_best_submission.csv',index=False)

In [36]:
import pandas as pd

In [ ]:
train=pd.read_csv()